# Bibliotecas

In [1]:
import pandas as pd

# Limpeza dos dados

## Lendo os arquivos

In [2]:
#produto = "celulares"
#produto = "notebooks"
#produto = "tvs"
#produto = "geladeiras"
produto = "fogoes"

In [3]:
df_americanas = pd.read_csv(f"Dados/Produtos/americanas_produtos_{produto}.csv", dtype = {'ean': str})
df_amazon = pd.read_csv(f"Dados/Produtos/amazon_produtos_{produto}.csv", dtype = {'ean': str})

In [ ]:
df_amazon["ean"].value_counts()

## Dropando Produtos sem EAN

In [4]:
print(f"Amazon: {df_amazon.shape}\nAmericanas: {df_americanas.shape}")

Amazon: (923, 6)
Americanas: (1656, 6)


In [5]:
df_americanas = df_americanas[df_americanas["ean"].notnull()]
df_amazon = df_amazon[df_amazon["ean"].notnull()]

In [ ]:
print(f"Amazon: {df_amazon.shape}\nAmericanas: {df_americanas.shape}")

## Adicionando a coluna do título com caixa baixa

In [6]:
df_amazon['titulo_cb'] = df_amazon['titulo'].str.lower()
df_americanas['titulo_cb'] = df_americanas['titulo'].str.lower()

## Retirando títulos com palavras indesejadas

In [ ]:
df_americanas["descricao"] = df_americanas["descricao"].apply(str)
df_americanas = df_americanas[~df_americanas["descricao"].str.contains('Preço especial só pra essa semana!!')]

In [ ]:
'''bad_words = ["acessório", "acessorio", "adaptador", "alargadores", "anel", "arduino", "atomiser", "avatar",
             "aparelho de som", "automotivo", "axiom",
             "bandeja", "biometria", "bike", "boroscópio", "box", 
             "cable", "cabo", "caixa", "campainha", "calculator", "calculadora", "capa", "capinha", "card",
             "case", "caso", "carregador", "carro"
             "chassi", "chaveiro", "chave seletora", "clip",
             "coletor", "collar", "conjunto", "comoda", "cômoda", "cordão", "cortador", "cosplay",
             "drone", "eon", "eixo", "eixos", "espelho", "filme", "franquia", "gancho", "grânulos", "gaveta",
             "handmade", "hemobblo", "hemobllo", "hd externo", "jbl", "junta", "kit",
             "lente", "lixa", "lurrose",
             "m-disc", "mágico", "magico", "mesa", "microsd", "monitor", "montagem", "moochy",
             "netcna", "óculos", "oculos", "ornament",
             "papel", "parede", "part", "partes", "pedal", "pen drive", "pendrive", "pneu", "pneus", "power bank",
             "prato", "prettyzoom", "pretyzoom", "proteção", "protetor", "pulseira",
             "raspberry", "radiador", "rank de memória", "realidade virtual", "refrigerador", "regeneradora",
             "relogio", "relógio", "remoção", "reparo", "reposição", "reposiçao", "resfriador", "retrovisor", "roda",
             "ronyme", "rv", "scanner", "sensor", "smart tv", "soquete", "stick", "streaming box", "suporte",
             "tablet", "tampa", "tomada", "tomadas", "tradutor", "turbopower",
             "ultra-sensível", "upgrade", "upgrades",
             "vador", "veado", "vedação", "verbatim", "vidro", "vr", "xilofone", "xylophone",
             "watch", "zawa", "zbox"
            ]'''

bad_words = ["caminhão", "caminhao", "onibus", "ônibus", "van", "lancha", "barco", "volkswagen", "motor home", "motorhome"]

print(f"Amazon: {df_amazon.shape}\nAmericanas: {df_americanas.shape}")

df_amazon_ft = df_amazon[~df_amazon["titulo_cb"].str.contains('|'.join(bad_words))]
df_americanas_ft = df_americanas[~df_americanas["titulo_cb"].str.contains('|'.join(bad_words))]
print(f"\nFiltrado:\nAmazon: {df_amazon_ft.shape}\nAmericanas: {df_americanas_ft.shape}")

In [ ]:
#df_amazon_ft['url'].loc[970]

In [ ]:
#df_americanas_ft[350:].head(50)

## Encontrando EANs repetidos na mesma loja

### Removendo produtos com o mesmo título

In [7]:
df_amazon_ft = pd.DataFrame(df_amazon.loc[~df_amazon["titulo"].duplicated(keep = 'first')])
df_americanas_ft = pd.DataFrame(df_americanas.loc[~df_americanas["titulo"].duplicated(keep = 'first')])

#df_amazon_ft = pd.DataFrame(df_amazon_ft.loc[~df_amazon_ft["titulo"].duplicated(keep = 'first')])
#df_americanas_ft = pd.DataFrame(df_americanas_ft.loc[~df_americanas_ft["titulo"].duplicated(keep = 'first')])

In [ ]:
print(f"Amazon: {df_amazon_ft.shape}\nAmericanas: {df_americanas_ft.shape}")

### Verificando Títulos de EANs duplicados

In [8]:
ama_vc_ean = df_amazon_ft[df_amazon_ft["ean"].notnull()]["ean"].value_counts()
ama_duplicados = ama_vc_ean[ama_vc_ean > 1].index[0]
df_amazon_ft[df_amazon_ft['ean'] == ama_duplicados]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [10]:
ame_vc_ean = df_americanas_ft[df_americanas_ft["ean"].notnull()]["ean"].value_counts()
ame_duplicados = ame_vc_ean[ame_vc_ean > 1].index
df_americanas_ft[df_americanas_ft["ean"].isin(ame_duplicados)].sort_values(by = ['ean']).head(1)

,categoria,descricao,ean,preco,titulo,url,titulo_cb,loja
759,/categoria/eletrodomesticos/fogao/fogao-de-piso,Descrição: Design super moderno Mesa de vidro ...,7890170734149,"R$ 1.450,00",Fogão de Piso Suggar Cook Mesa de Vidro Front ...,https://www.americanas.com.br/produto/4867174556,fogão de piso suggar cook mesa de vidro front ...,americanas


In [9]:
df_americanas_ft['loja'] = "americanas"
df_amazon_ft['loja'] = "amazon"

In [11]:
frames = [df_americanas_ft, df_amazon_ft]
df_final = pd.concat(frames, ignore_index = True)

In [12]:
df_final_ean = df_final[df_final["ean"].notnull()]
df_final_ean.shape

(1414, 8)

In [13]:
#df_final_ean.to_csv(f"Dados/Produtos Processados/pp_{produto}.csv", index = False)

In [ ]:
vc_ean = df_final[df_final["ean"].notnull()]["ean"].value_counts()
duplicados = vc_ean[vc_ean > 1].index
df_final_dup = df_final[df_final["ean"].isin(duplicados)].sort_values(by = ['ean'])

## Fazendo matches

In [14]:
def criar_dicionario(descricao_1, ean_1, titulo_1, url_1, titulo_cb_1, loja_1,
                     descricao_2, ean_2, titulo_2, url_2, titulo_cb_2, loja_2,
                     match):
    
    return {'descricao_1' : descricao_1 , 'ean_1' : ean_1, 'titulo_1' : titulo_1,
            'url_1' : url_1, 'titulo_cb_1' : titulo_cb_1, 'loja_1' : loja_1,
            
            'descricao_2' : descricao_2 , 'ean_2' : ean_2, 'titulo_2' : titulo_2,
            'url_2' : url_2 , 'titulo_cb_2' : titulo_cb_2, 'loja_2' : loja_2,
            
            'match' : match
           }

In [15]:
def criar_pares(colunas, df_final_dup):

    df_pares = pd.DataFrame(columns = colunas)
    df_pares

    lista_ean = []

    for i in range(len(df_final_dup)):
        
        if df_final_dup.iloc[i]['ean'] not in lista_ean:
            
            lista_ean.append(df_final_dup.iloc[i]['ean'])
            
            indice = df_final_dup.index[df_final_dup['ean'] == df_final_dup.iloc[i]['ean']].tolist()
            
            for j in range(len(indice)-1):
                
                for k in range(j + 1, len(indice)):
                    
                    dicionario = criar_dicionario(descricao_1  = df_final_dup.loc[indice[j]]['descricao'] , ean_1 = df_final_dup.loc[indice[j]]['ean'],
                                                  titulo_1 = df_final_dup.loc[indice[j]]['titulo'], url_1 = df_final_dup.loc[indice[j]]['url'],
                                                  titulo_cb_1 = df_final_dup.loc[indice[j]]['titulo_cb'], loja_1 = df_final_dup.loc[indice[j]]['loja'],
                                                  
                                                  descricao_2 = df_final_dup.loc[indice[k]]['descricao'] , ean_2 = df_final_dup.loc[indice[k]]['ean'],
                                                  titulo_2 = df_final_dup.loc[indice[k]]['titulo'], url_2 = df_final_dup.loc[indice[k]]['url'],
                                                  titulo_cb_2 = df_final_dup.loc[indice[k]]['titulo_cb'], loja_2 = df_final_dup.loc[indice[k]]['loja'],
                                                  
                                                  match = 1
                                                  )
                
                    df_pares = df_pares.append(dicionario, ignore_index = True)

    return df_pares

In [16]:
colunas = ["descricao_1", "ean_1", "titulo_1", "url_1",	"titulo_cb_1", "loja_1",
           "descricao_2", "ean_2", "titulo_2", "url_2",	"titulo_cb_2", "loja_2",
           "match"]

In [ ]:
df_pares = criar_pares(colunas, df_final_dup)

In [ ]:
#df_pares[df_pares['ean_1']=="6934177738180"][['descricao_1', 'ean_1', 'titulo_1', 'descricao_2', 'ean_2', 'titulo_2', 'loja_1', 'loja_2', 'url_1', 'url_2']]
#df_pares[['descricao_1', 'ean_1', 'titulo_1', 'descricao_2', 'ean_2', 'titulo_2', 'loja_1', 'loja_2', 'url_1', 'url_2']]

In [ ]:
df_pares

## Salvando Pares

In [ ]:
n_matches = df_pares.shape[0]*5
df_nao_pares = pd.DataFrame(columns = colunas)

for i in range(n_matches):
    created_match = False
    
    while(not created_match):
        df_sorteado = df_final_ean.sample(n = 2)
        if(df_sorteado.iloc[0]['ean']!= df_sorteado.iloc[1]['ean']):
            
            dic = criar_dicionario(df_sorteado.iloc[0]['descricao'], df_sorteado.iloc[0]['ean'], df_sorteado.iloc[0]['titulo'], df_sorteado.iloc[0]['url'], df_sorteado.iloc[0]['titulo_cb'], df_sorteado.iloc[0]['loja'],
                                   df_sorteado.iloc[1]['descricao'], df_sorteado.iloc[1]['ean'], df_sorteado.iloc[1]['titulo'], df_sorteado.iloc[1]['url'], df_sorteado.iloc[1]['titulo_cb'], df_sorteado.iloc[1]['loja'],
                                   0)
            
            df_nao_pares = df_nao_pares.append(dic, ignore_index = True)
            created_match = True

Salvando pares não matches

In [ ]:
df_nao_pares.to_csv(f"Dados/Não Pares/nao_pares_{produto}.csv", index = False)

## Gerando dataset final

In [ ]:
df_pares = pd.read_csv(f"Dados/Pares/pares_{produto}.csv")
df_nao_pares = pd.read_csv(f"Dados/Não Pares/nao_pares_{produto}.csv")

df_pares_total = pd.concat([df_pares, df_nao_pares], ignore_index = True)

In [ ]:
df_pares_total = df_pares_total.sample(frac = 1)
df_pares_total.to_csv(f"Dados/Total Pares/total_pares_{produto}.csv", index = False)

In [ ]:
df_pares_total.shape

In [ ]:
df_pares_total

In [ ]:
df_americanas_ft[df_americanas_ft["ean"].notnull()]["ean"].value_counts()

# Soft Negative

In [27]:
NUM_AMOSTRAS = 280
N_C = 298
N2 = 399
N3 = 280
N4 = 911
N5 = 327

In [28]:
lista_nomes = ["celulares", "notebooks", "geladeiras", "fogoes", "tvs"]

In [29]:
lista_df = []
for nome in lista_nomes:
    
    df = pd.read_csv(f"Dados/Produtos Processados/pp_{nome}.csv")
    lista_df.append(df)

In [30]:
df_final_hn = pd.concat(lista_df, ignore_index = True)

In [33]:
#n_matches = len(lista_nomes)*5
n_matches = 11075
df_nao_pares = pd.DataFrame(columns = colunas)

for i in range(n_matches):
    created_match = False
    
    while(not created_match):
        df_sorteado = df_final_hn.sample(n = 2)
        if(df_sorteado.iloc[0]['ean']!= df_sorteado.iloc[1]['ean']):
            
            dic = criar_dicionario(df_sorteado.iloc[0]['descricao'], df_sorteado.iloc[0]['ean'], df_sorteado.iloc[0]['titulo'], df_sorteado.iloc[0]['url'], df_sorteado.iloc[0]['titulo_cb'], df_sorteado.iloc[0]['loja'],
                                   df_sorteado.iloc[1]['descricao'], df_sorteado.iloc[1]['ean'], df_sorteado.iloc[1]['titulo'], df_sorteado.iloc[1]['url'], df_sorteado.iloc[1]['titulo_cb'], df_sorteado.iloc[1]['loja'],
                                   0)
            
            df_nao_pares = df_nao_pares.append(dic, ignore_index = True)
            created_match = True

C:\Users\Amamiya\AppData\Local\Temp\ipykernel_9020\2646649132.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_nao_pares = df_nao_pares.append(dic, ignore_index = True)
C:\Users\Amamiya\AppData\Local\Temp\ipykernel_9020\2646649132.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_nao_pares = df_nao_pares.append(dic, ignore_index = True)
C:\Users\Amamiya\AppData\Local\Temp\ipykernel_9020\2646649132.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_nao_pares = df_nao_pares.append(dic, ignore_index = True)
C:\Users\Amamiya\AppData\Local\Temp\ipykernel_9020\2646649132.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  d

In [34]:
df_nao_pares.to_csv(f"Dados/Não Pares/nao_pares_pp_desbalanceado.csv", index = False)